### install & download

In [1]:
!pip install HanTa

     |████████████████████████████████| 15.0 MB 4.7 MB/s eta 0:00:01


In [2]:
!pip install transformers

     |████████████████████████████████| 7.9 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 197 kB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 441 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.6 MB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 311 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 166 kB 4.6 MB/s eta 0:00:01


In [3]:
!pip3 install wordfreq

     |████████████████████████████████| 56.8 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 296 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 234 kB 3.1 MB/s eta 0:00:01
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.5
    Uninstalling wcwidth-0.2.5:
      Successfully uninstalled wcwidth-0.2.5
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2


In [4]:
!python -m spacy download de_core_news_md

     |████████████████████████████████| 44.4 MB 1.9 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_md')


In [5]:
!python -m spacy download ru_core_news_md

     |████████████████████████████████| 41.9 MB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 1.5 MB/s eta 0:00:01
  Using cached docopt-0.6.2.tar.gz (25 kB)
     |████████████████████████████████| 8.2 MB 5.4 MB/s eta 0:00:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=70be8b4d33bbee220cac6fd2027c8329bff83dc3f5654aa547cecdc255b0dcbe
  Stored in directory: /Users/galina.ryazanskaya/Library/Caches/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [6]:
!wget https://int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com/vectors.txt

--2023-11-29 19:14:18--  https://int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com/vectors.txt
Resolving int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com (int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com)... 52.219.75.76, 3.5.135.12, 52.219.75.128, ...
Connecting to int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com (int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com)|52.219.75.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3744610526 (3,5G) [text/plain]
Saving to: ‘vectors.txt’

vectors.txt         100%[===================>]   3,49G  5,24MB/s    in 11m 56s 

2023-11-29 19:26:14 (4,99 MB/s) - ‘vectors.txt’ saved [3744610526/3744610526]



### import & load

https://github.com/aalok-sathe/surprisal
does not support bert

In [7]:
import attr

import numpy as np
import networkx as nx
import nltk
import nltk.stem
import re
import spacy
import torch

from collections import Counter
from functools import cached_property, cache
from HanTa import HanoverTagger as ht
from nltk.corpus import stopwords
from spacy.lang.de.examples import sentences
from transformers import AutoModel, AutoTokenizer
from transformers import BertTokenizer, BertForNextSentencePrediction
from typing import Callable, Dict, Optional, List
from wordfreq import word_frequency

ModuleNotFoundError: No module named 'torch'

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

In [ ]:
nlp_de = spacy.load("de_core_news_md")
nlp_ru = spacy.load("ru_core_news_md")

In [10]:
stopwords_ru = stopwords.words("russian")

In [11]:
stopwords_de = stopwords.words("german")

In [ ]:
#german_stopwords = ["a","ab","aber","ach","acht","achte","achten","achter","achtes","ag","alle","allein","allem","allen","aller","allerdings","alles","allgemeinen","als","also","am","an","ander","andere","anderem","anderen","anderer","anderes","anderm","andern","anderr","anders","au","auch","auf","aus","ausser","ausserdem","außer","außerdem","b","bald","bei","beide","beiden","beim","beispiel","bekannt","bereits","besonders","besser","besten","bin","bis","bisher","bist","c","d","d.h","da","dabei","dadurch","dafür","dagegen","daher","dahin","dahinter","damals","damit","danach","daneben","dank","dann","daran","darauf","daraus","darf","darfst","darin","darum","darunter","darüber","das","dasein","daselbst","dass","dasselbe","davon","davor","dazu","dazwischen","daß","dein","deine","deinem","deinen","deiner","deines","dem","dementsprechend","demgegenüber","demgemäss","demgemäß","demselben","demzufolge","den","denen","denn","denselben","der","deren","derer","derjenige","derjenigen","dermassen","dermaßen","derselbe","derselben","des","deshalb","desselben","dessen","deswegen","dich","die","diejenige","diejenigen","dies","diese","dieselbe","dieselben","diesem","diesen","dieser","dieses","dir","doch","dort","drei","drin","dritte","dritten","dritter","drittes","du","durch","durchaus","durfte","durften","dürfen","dürft","e","eben","ebenso","ehrlich","ei","ei,","eigen","eigene","eigenen","eigener","eigenes","ein","einander","eine","einem","einen","einer","eines","einig","einige","einigem","einigen","einiger","einiges","einmal","eins","elf","en","ende","endlich","entweder","er","ernst","erst","erste","ersten","erster","erstes","es","etwa","etwas","euch","euer","eure","eurem","euren","eurer","eures","f","folgende","früher","fünf","fünfte","fünften","fünfter","fünftes","für","g","gab","ganz","ganze","ganzen","ganzer","ganzes","gar","gedurft","gegen","gegenüber","gehabt","gehen","geht","gekannt","gekonnt","gemacht","gemocht","gemusst","genug","gerade","gern","gesagt","geschweige","gewesen","gewollt","geworden","gibt","ging","gleich","gott","gross","grosse","grossen","grosser","grosses","groß","große","großen","großer","großes","gut","gute","guter","gutes","h","hab","habe","haben","habt","hast","hat","hatte","hatten","hattest","hattet","heisst","her","heute","hier","hin","hinter","hoch","hätte","hätten","i","ich","ihm","ihn","ihnen","ihr","ihre","ihrem","ihren","ihrer","ihres","im","immer","in","indem","infolgedessen","ins","irgend","ist","j","ja","jahr","jahre","jahren","je","jede","jedem","jeden","jeder","jedermann","jedermanns","jedes","jedoch","jemand","jemandem","jemanden","jene","jenem","jenen","jener","jenes","jetzt","k","kam","kann","kannst","kaum","kein","keine","keinem","keinen","keiner","keines","kleine","kleinen","kleiner","kleines","kommen","kommt","konnte","konnten","kurz","können","könnt","könnte","l","lang","lange","leicht","leide","lieber","los","m","machen","macht","machte","mag","magst","mahn","mal","man","manche","manchem","manchen","mancher","manches","mann","mehr","mein","meine","meinem","meinen","meiner","meines","mensch","menschen","mich","mir","mit","mittel","mochte","mochten","morgen","muss","musst","musste","mussten","muß","mußt","möchte","mögen","möglich","mögt","müssen","müsst","müßt","n","na","nach","nachdem","nahm","natürlich","neben","nein","neue","neuen","neun","neunte","neunten","neunter","neuntes","nicht","nichts","nie","niemand","niemandem","niemanden","noch","nun","nur","o","ob","oben","oder","offen","oft","ohne","ordnung","p","q","r","recht","rechte","rechten","rechter","rechtes","richtig","rund","s","sa","sache","sagt","sagte","sah","satt","schlecht","schluss","schon","sechs","sechste","sechsten","sechster","sechstes","sehr","sei","seid","seien","sein","seine","seinem","seinen","seiner","seines","seit","seitdem","selbst","sich","sie","sieben","siebente","siebenten","siebenter","siebentes","sind","so","solang","solche","solchem","solchen","solcher","solches","soll","sollen","sollst","sollt","sollte","sollten","sondern","sonst","soweit","sowie","später","startseite","statt","steht","suche","t","tag","tage","tagen","tat","teil","tel","tritt","trotzdem","tun","u","uhr","um","und","uns","unse","unsem","unsen","unser","unsere","unserer","unses","unter","v","vergangenen","viel","viele","vielem","vielen","vielleicht","vier","vierte","vierten","vierter","viertes","vom","von","vor","w","wahr","wann","war","waren","warst","wart","warum","was","weg","wegen","weil","weit","weiter","weitere","weiteren","weiteres","welche","welchem","welchen","welcher","welches","wem","wen","wenig","wenige","weniger","weniges","wenigstens","wenn","wer","werde","werden","werdet","weshalb","wessen","wie","wieder","wieso","will","willst","wir","wird","wirklich","wirst","wissen","wo","woher","wohin","wohl","wollen","wollt","wollte","wollten","worden","wurde","wurden","während","währenddem","währenddessen","wäre","würde","würden","x","y","z","z.b","zehn","zehnte","zehnten","zehnter","zehntes","zeit","zu","zuerst","zugleich","zum","zunächst","zur","zurück","zusammen","zwanzig","zwar","zwei","zweite","zweiten","zweiter","zweites","zwischen","zwölf","über","überhaupt","übrigens"]

## GloVe German

In [12]:
punct = '\!"#$%&\'\(\)\*\+,\-\./:;<=>\?@[\\]^_`{|}~\^„“/…'
def test_unicode(text):
    non_unicode = re.findall(f'[^A-Za-zÀ-ž0-9\\n {punct}]', text)
    return not bool(non_unicode), non_unicode

In [ ]:
with open('vectors.txt') as f:
    lines = f.readlines()

In [ ]:
def process_line(line):
    key, str_value = line.strip().split(' ', 1)
    return key, np.array([float(x) for x in str_value.split()])

In [ ]:
glove = {}
for line in lines:
    key, value = process_line(line)
    if test_unicode(key):
        glove[key] = value

In [ ]:
glove_oov = set()
def get_glove_vector(word):
    if word in glove:
        return glove[word]
    else:
        glove_oov.add(word)
        return None

#### Save glove

In [ ]:
import gensim
from tqdm import tqdm

In [45]:

def save_word2vec_format(fname, vocab, vector_size, binary=True):
  # gensim 3.4
    """Store the input-hidden weight matrix in the same format used by the original
    C word2vec-tool, for compatibility.

    Parameters
    ----------
    fname : str
        The file path used to save the vectors in.
    vocab : dict
        The vocabulary of words.
    vector_size : int
        The number of dimensions of word vectors.
    binary : bool, optional
        If True, the data wil be saved in binary word2vec format, else it will be saved in plain text.


    """

    total_vec = len(vocab)
    with gensim.utils.open(fname, 'wb') as fout:
        print(total_vec, vector_size)
        fout.write(gensim.utils.to_utf8("%s %s\n" % (total_vec, vector_size)))
        # store in sorted order: most frequent words at the top
        for word, row in tqdm(vocab.items()):
            if binary:
                row = row.astype(np.float32)
                fout.write(gensim.utils.to_utf8(word) + b" " + row.tostring())
            else:
                fout.write(gensim.utils.to_utf8("%s %s\n" % (word, ' '.join(repr(val) for val in row))))

In [46]:
%%time
# save_word2vec_format(binary=True, fname='/content/drive/MyDrive/studies/science/Schizophrenia_private/german/glove_de.bin', vocab=glove, vector_size=300)

1309281 300


  0%|          | 0/1309281 [00:00<?, ?it/s]<ipython-input-45-e1f2e0282abc>:27: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  fout.write(gensim.utils.to_utf8(word) + b" " + row.tostring())
100%|██████████| 1309281/1309281 [02:40<00:00, 8165.24it/s]

CPU times: user 8.46 s, sys: 2 s, total: 10.5 s
Wall time: 2min 40s


In [ ]:
# model = gensim.models.KeyedVectors.load_word2vec_format('glove_de.bin', binary=True)

## Preprocess
https://textmining.wp.hs-hannover.de/Preprocessing.html

https://spacy.io/usage/linguistic-features

In [ ]:
#from pymystem3 import Mystem
#mystem = Mystem()
# def preprocess_text_ru(text):
#     tokens = mystem.lemmatize(text.lower())
#     tokens = [token for token in tokens if token not in stopwords_ru\
#               and token != " " \
#               and token.strip() not in punct]

#     text = " ".join(tokens)

#     return text

In [48]:
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

In [49]:
default_sent_tokenizer = lambda t: nltk.sent_tokenize(t, 'german')
default_word_tokenizer = lambda t: nltk.word_tokenize(t, 'german')
# default_stemmer = lambda w: nltk.stem.snowball.GermanStemmer().stem(w)
default_pos_tagger = lambda w: tagger.analyze(w)[-1]
default_lemmatizer = lambda w: tagger.analyze(w)[0]
default_model = lambda w: get_glove_vector(w)

In [13]:
def idf_sent_vectors(words: List[str], vectors: List[np.array]) -> np.array:
  assert len(words) == len(vectors)
  weights = [word_frequency(w, 'de') for w in words]
  return np.average(vectors, axis=0, weights=weights)

In [14]:
@attr.s(auto_attribs=True)
class TextData():
  text: str
  words: Optional[List[str]] = attr.ib(factory=list)
  lemmas: Optional[List[str]] = attr.ib(factory=list)
  pos: Optional[List[str]] = attr.ib(factory=list)
  sents: Optional[List[str]] = attr.ib(factory=list)
  sent_words: Optional[List[List[str]]] = attr.ib(factory=list)
  word_vectors: Optional[List[np.array]] = attr.ib(factory=list)
  word_for_vectors: Optional[List[str]] = attr.ib(factory=list)
  sent_word_vectors: Optional[List[List[np.array]]] = attr.ib(factory=list)
  sent_word_for_vectors: Optional[List[List[str]]] = attr.ib(factory=list)
  oov: Optional[List[str]] = attr.ib(factory=list)

  def __attrs_post_init__(self):
    self.sent_vectors = [idf_sent_vectors(words, vectors) for words, vectors in zip(self.sent_word_for_vectors, self.sent_word_vectors)]

In [52]:
def process_default(text: str, stopwords: Optional[List[str]] = []) -> TextData:
    words = default_word_tokenizer(text)
    sents = default_sent_tokenizer(text)
    sent_words = [default_word_tokenizer(sent) for sent in sents]
    pos = [default_pos_tagger(w) for w in words]
    lemmas = [default_lemmatizer(w) for w in words]
    word_vectors = [default_model(w.lower()) for w in lemmas if default_model(w.lower()) is not None]
    word_for_vectors = [w for w in lemmas if default_model(w.lower()) is not None]
    oov = [w for w in lemmas if default_model(w.lower()) is None]
    sent_word_vectors = [[default_model(default_lemmatizer(w).lower()) \
                          for w in sent if \
                          default_model(default_lemmatizer(w).lower()) is not None]
                         for sent in sent_words]
    sent_word_for_vectors = [[default_lemmatizer(w) for w in sent if \
                              default_model(default_lemmatizer(w).lower()) \
                              is not None]
                              for sent in sent_words]
    data = TextData(text=text, words=words, lemmas=lemmas, pos=pos, sents=sents,
                    sent_words=sent_words, word_vectors=word_vectors, oov=oov,
                    sent_word_vectors=sent_word_vectors,
                    word_for_vectors=word_for_vectors,
                    sent_word_for_vectors=sent_word_for_vectors)
    return data

In [15]:
def process_spacy(text: str, stopwords: Optional[List[str]] = [], nlp=nlp_de) -> TextData:
    doc = nlp(text)
    words = [token for token in doc if token.text not in stopwords]
    sents = [str(sent) for sent in doc.sents]
    sent_words = [[token for token in nlp(sent) if token.text not in stopwords]
                   for sent in sents]
    sent_words = [s for s in sent_words if s]
    pos = [token.pos_ for token in doc]
    lemmas = [token.lemma_ for token in doc]
    word_vectors = [token.vector for token in doc
                    if ((token.text not in stopwords
                         and not token.is_oov
                         and token.pos_ not in ['PUNCT', 'NUM']))]
    word_for_vectors = [token.text for token in doc
                        if ((token.text not in stopwords
                              and not token.is_oov
                              and token.pos_ not in ['PUNCT', 'NUM']))]
    oov = list(set(token.text for token in doc
                   if (token.text not in stopwords
                       and token.is_oov
                       and token.pos_ not in ['PUNCT', 'NUM'])))
    sent_word_vectors = [[token.vector for token in nlp(sent)
                        if ((token.text not in stopwords
                            and not token.is_oov
                            and token.pos_ not in ['PUNCT', 'NUM']))] for sent in sents]
    sent_word_vectors = [s for s in sent_word_vectors if s]
    sent_word_for_vectors = [[token.text for token in nlp(sent)
                              if ((token.text not in stopwords
                                  and not token.is_oov
                                  and token.pos_ not in ['PUNCT', 'NUM']))] for sent in sents]
    sent_word_for_vectors = [s for s in sent_word_for_vectors if s]
    data = TextData(text=text, words=words, lemmas=lemmas, pos=pos, sents=sents,
                sent_words=sent_words, word_vectors=word_vectors, oov=oov,
                sent_word_vectors=sent_word_vectors,
                word_for_vectors=word_for_vectors,
                sent_word_for_vectors=sent_word_for_vectors)
    return data

In [16]:
some_large_text = "Der Dackel zeichnet sich durch niedrige, kurzläufige, langgestreckte, aber kompakte Gestalt aus. Er ist sehr muskulös, mit aufrechter Haltung des Kopfes und aufmerksamem Gesichtsausdruck. Die verkürzten Beine der Dackel sind das Resultat einer gezielten Selektion auf Chondrodysplasie und sind im Rassestandard verankert. Es gibt die Hunde in verschiedenen Größen und Fellvariationen: Langhaar, Rauhaar, Kurzhaar in jeweils vielen Farbvariationen, auch mehrfarbig, gestromt oder gefleckt. Während gefleckte Tiere als eine der 4 Färbungen durch den FCI definiert sind, werden Schwarze Tiere ohne Brand sowie weiße mit oder ohne Brand im Rassestandard der FCI ausdrücklich ausgeschlossen[2]. Unter AKC-Regeln können weiß gescheckte Dackel ausgestellt werden und sind dort als Piebalds bekannt.[3] Die hängenden Ohren sind nicht zu weit vorne angesetzt, ausreichend, aber nicht übertrieben lang und abgerundet. In den drei Haararten werden die Dackel im FCI-Standard nach ihrer Größe unterschieden in Teckel (T) (früher Normalteckel), Brustumfang (BU) über 35 cm, Gewichtsobergrenze etwa 9 kg, Zwergteckel (Zw), BU über 30 bis 35 cm, und Kaninchenteckel (Kt), BU bis 30 cm."

In [ ]:
# data_default = process_default(some_large_text)

In [17]:
data = process_spacy(some_large_text)

In [18]:
some_large_text_ru = "Знаете ли вы? Будапештский университет иудаики, около 1890 года Социалистическая власть не мешала работе раввинской семинарии (на илл.). Будапештский университет иудаики был открыт в 1877 году, через несколько десятилетий после того, как в Падуе, Меце, Париже и Бреслау были построены первые европейские раввинские семинарии. Тем не менее, он остаётся старейшим существующим учебным заведением в мире, где готовят раввинов."

In [19]:
data_ru = process_spacy(some_large_text_ru, nlp=nlp_ru)

## Lexical

In [20]:
def TTR(words: List[str]) -> float:
  return len(set(words)) / len(words)

In [21]:
TTR(data.lemmas)

0.5643564356435643

In [22]:
def MATTR(words: List[str], w: Optional[int] = 10) -> float:
  if len(words) <= w:
    return TTR(words)
  else:
    ttrs = []
    for i in range(len(words)-w):
      ttrs.append(TTR(words[i:i+w]))
    return np.mean(ttrs)

In [23]:
MATTR(data.lemmas)

0.8822916666666668

## Syntactic

In [24]:
def pos_rates(pos: List[str]) -> float:
  return {k: v/len(pos) for k, v in Counter(pos).items()}

In [65]:
pos_rates(data.pos)

{'DET': 0.06930693069306931,
 'NOUN': 0.18316831683168316,
 'VERB': 0.04455445544554455,
 'PRON': 0.024752475247524754,
 'ADP': 0.09900990099009901,
 'ADJ': 0.07425742574257425,
 'PUNCT': 0.18316831683168316,
 'CCONJ': 0.0594059405940594,
 'AUX': 0.04950495049504951,
 'ADV': 0.08415841584158416,
 'PROPN': 0.06930693069306931,
 'SCONJ': 0.0049504950495049506,
 'NUM': 0.034653465346534656,
 'PART': 0.01485148514851485,
 'X': 0.0049504950495049506}

In [25]:
def sent_lengts(sents_words: List[List[str]]) -> float:
  return np.mean([len(sent) for sent in sents_words])

In [67]:
sent_lengts(data.sent_words)

28.857142857142858

## Graph

In [26]:
class _graphStatistics():

	def __init__(self, graph: nx.MultiDiGraph):
		self.graph = graph

	def statistics(self) -> Dict[str, float]:
		res = {}
		graph = self.graph
		res['number_of_nodes'] = graph.number_of_nodes()
		res['number_of_edges'] = graph.number_of_edges()
		res['PE'] =  (np.array(list(Counter(graph.edges()).values()))>1).sum()
		res['LCC'] = len(max(nx.weakly_connected_components(graph), key=len))
		res['LSC'] = len(max(nx.strongly_connected_components(graph), key=len))

		degrees = [v for k, v in graph.degree()]
		res['degree_average'] =  np.mean(degrees)
		res['degree_std'] =  np.std(degrees)


		adj_matrix = nx.adjacency_matrix(graph).toarray()
		adj_matrix2 = np.dot(adj_matrix , adj_matrix)
		adj_matrix3 = np.dot(adj_matrix2 , adj_matrix)

		res['L1'] =  np.trace(adj_matrix)
		res['L2'] =  np.trace(adj_matrix2)
		res['L3'] =  np.trace(adj_matrix3)

		return res


class naiveGraph():
	def __init__(self):
		pass

	def _text2graph(self, words: List[str]) -> nx.MultiDiGraph:
		gr = nx.MultiDiGraph()
		gr.add_edges_from(zip(words[:-1], words[1:]))
		return gr

	def analyzeText(self, text: str) -> Dict[str, float]:
		dgr = self._text2graph(text)
		return _graphStatistics(dgr).statistics()

# https://github.com/facuzeta/speechgraph/blob/master/speechgraph/speechgraph.py

In [69]:
naiveGraph().analyzeText(data.lemmas)

{'number_of_nodes': 114,
 'number_of_edges': 201,
 'PE': 10,
 'LCC': 114,
 'LSC': 114,
 'degree_average': 3.526315789473684,
 'degree_std': 6.730671516139514,
 'L1': 4,
 'L2': 44,
 'L3': 271}

In [27]:
def moving_graph_statistics(words: List[str], w: Optional[int] = 10) -> Dict[str, float]:
  if len(words) <= w:
    return naiveGraph().analyzeText(words)
  else:
    graph_stats = {}
    for i in range(len(words)-w):
      current_stats = naiveGraph().analyzeText(words[i:i+w])
      graph_stats = {key: graph_stats.get(key, []) + [value] for key, value in current_stats.items()}

    return {key: np.mean(value) for key, value in graph_stats.items()}

In [71]:
moving_graph_statistics(data.lemmas, 15)

{'number_of_nodes': 12.732620320855615,
 'number_of_edges': 14.0,
 'PE': 0.10695187165775401,
 'LCC': 12.732620320855615,
 'LSC': 6.705882352941177,
 'degree_average': 2.233913798619681,
 'degree_std': 1.1645017125950392,
 'L1': 0.25133689839572193,
 'L2': 1.7379679144385027,
 'L3': 2.9946524064171123}

## LM

In [28]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [29]:
def cos_sim(v1, v2):
    return np.inner(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))

In [30]:
def get_local_coherence_list(clause_vectors: List[np.array]) -> List[float]:
    local_coherence_list = []
    for i in range(len(clause_vectors)-1):
        local_coherence_list.append(cos_sim(clause_vectors[i], clause_vectors[i+1]))
    return local_coherence_list

In [31]:
def get_second_order_coherence_list(clause_vectors: List[np.array]) -> List[float]:
    SOC_list = []
    for i in range(len(clause_vectors)-2):
        SOC_list.append(cos_sim(clause_vectors[i], clause_vectors[i+2]))
    return SOC_list

In [32]:
def compare_text_to_a_standard_vector(clause_vectors: List[np.array], standard_vector: np.array) -> float:
    average_file_vector = np.average(clause_vectors, axis=0)
    return cos_sim(average_file_vector, standard_vector)

In [33]:
def get_global_coherence_list(clause_vectors: List[np.array]) -> List[float]:
    standard_vector = np.average(clause_vectors, axis=0)
    return [cos_sim(vec, standard_vector) for vec in clause_vectors]

In [34]:
def get_cumulative_global_coherence_list(clause_vectors: List[np.array]) -> List[float]:
    cumulative_global_coherence_list = []
    for i in range(len(clause_vectors)):
      cumulative_vector = np.average(clause_vectors[:i+1], axis=0)
      cumulative_global_coherence_list.append(cos_sim(clause_vectors[i], cumulative_vector))
    return cumulative_global_coherence_list

### BERT

https://aclanthology.org/2020.coling-main.598/

In [35]:
model_name_de = "bert-base-german-cased"
model_de = AutoModel.from_pretrained(model_name_de)
tokenizer_de = AutoTokenizer.from_pretrained(model_name_de)

In [36]:
model_name_ru = "DeepPavlov/rubert-base-cased"
model_ru = AutoModel.from_pretrained(model_name_ru)
tokenizer_ru = AutoTokenizer.from_pretrained(model_name_ru)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### sent vectorize

In [37]:
inputs = tokenizer_de(some_large_text, return_tensors="pt")
outputs, outputs_1 = model_de(**inputs).to_tuple()

In [38]:
inputs = tokenizer_ru(some_large_text_ru, return_tensors="pt")
outputs, outputs_1 = model_ru(**inputs).to_tuple()

In [41]:
def bert_sent_vectorize(text: str, model=model_de, tokenizer=tokenizer_de) -> np.array:
  inputs = tokenizer(text, return_tensors="pt")
  word_vectors, sent_vector = model(**inputs).to_tuple()
  return sent_vector.detach().squeeze().numpy()

In [43]:
bert_sent_vectorize(data.sents[0])[:10]

array([ 0.9998575 ,  0.09257472, -0.06688777,  0.4215318 , -0.4227732 ,
        0.9772717 , -0.18655269, -0.66038036,  0.17960058, -0.21039967],
      dtype=float32)

In [44]:
bert_sent_vectorize(data_ru.sents[0], model=model_ru, tokenizer=tokenizer_ru)[:10]

array([ 0.06534111,  0.02699744,  0.09295036,  0.22325635, -0.01652229,
        0.21227604,  0.05135136,  0.05099475, -0.07633962,  0.11428361],
      dtype=float32)

### next sentence prediction

In [45]:
tokenizer_nsp_de = BertTokenizer.from_pretrained(model_name_de)
model_nsp_de = BertForNextSentencePrediction.from_pretrained(model_name_de)
tokenizer_nsp_ru = BertTokenizer.from_pretrained(model_name_ru)
model_nsp_ru = BertForNextSentencePrediction.from_pretrained(model_name_ru)

In [46]:
def next_sent_prob(sent_text_1: str, sent_text_2: str,
                   tokenizer_nsp=tokenizer_nsp_de,
                   model_nsp=model_nsp_de) -> float:
  tokenized = tokenizer_nsp(sent_text_1, sent_text_2, return_tensors='pt')
  predict = model_nsp(**tokenized)
  pred = torch.nn.functional.softmax(predict.logits[0], dim=0)[0].item()
  return pred

In [59]:
sentence_A = "Ich bin nicht so sicher."
sentence_B = "Jedoch nur die Rasse Dachshund enthält."

In [60]:
next_sent_prob(sentence_A, sentence_B)

0.036054715514183044

In [57]:
sentence_A_ru = "Ну что такое?"
sentence_B_ru = "Почему-то все опять не работает."

In [58]:
next_sent_prob(sentence_A_ru, sentence_B_ru, tokenizer_nsp=tokenizer_nsp_ru, model_nsp=model_nsp_ru)

0.9802790880203247

In [55]:
def get_prob_list(sents: List[str], tokenizer_nsp=tokenizer_nsp_de, model_nsp=model_nsp_de) -> List[float]:
  return [next_sent_prob(sents[i], sents[i+1], tokenizer_nsp=tokenizer_nsp, model_nsp=model_nsp) for i in range(len(sents)-1)]

In [50]:
get_prob_list(data.sents)

[0.9973336458206177,
 0.9355976581573486,
 0.9985324144363403,
 0.9998193383216858,
 0.9998990297317505,
 0.9999611377716064]

In [56]:
get_prob_list(data_ru.sents, tokenizer_nsp=tokenizer_nsp_ru, model_nsp=model_nsp_ru)

[0.0777721181511879, 0.9999936819076538, 0.999984622001648]

## Apply

In [78]:
@attr.s(auto_attribs=True)
class ProcessTextData:
  data: TextData
  bert_lang: Optional[str] = attr.ib(default='de')

  def __attrs_post_init__(self):
    if self.bert_lang == 'de':
      self.bert_model = model_de
      self.bert_model_nsp = model_nsp_de
      self.bert_tokenizer = tokenizer_de
      self.bert_tokenizer_nsp = tokenizer_nsp_de
    elif self.bert_lang == 'ru':
      self.bert_model = model_ru
      self.bert_model_nsp = model_nsp_ru
      self.bert_tokenizer = tokenizer_ru
      self.bert_tokenizer_nsp = tokenizer_nsp_ru

    self.n_words = len(self.data.words)
    self.LTR = self.TTR()
    self.MALTR = self.MATTR()
    self.lexical = {'n_words': self.n_words,
                    'LTR': self.LTR,
                    'MALTR': self.MALTR}

    self.n_sents = len(self.data.sents)
    self.mean_sent_words = np.mean([len(s) for s in self.data.sent_words])
    self.pos_rates = pos_rates(self.data.pos)

    self.syntactic = {'n_sents': self.n_sents,
                    'mean_sent_words': self.mean_sent_words}
    self.syntactic.update(self.pos_rates)

    self.graph_statistics = self.graph_statistics()

    self.mean_lcoh = np.mean(self.local_coherence_list())
    self.mean_gcoh = np.mean(self.global_coherence_list())
    self.mean_cgcoh = np.mean(self.cumulative_global_coherence_list())
    self.mean_scoh = np.mean(self.second_order_coherence_list())
    self.mean_sent_prob = np.mean(self.sent_prob_list)

    self.LM = {'m_lcoh': self.mean_lcoh,
               'm_gcoh': self.mean_gcoh,
               'm_cgcoh': self.mean_cgcoh,
               'm_scoh': self.mean_scoh,
               'm_sporb': self.mean_sent_prob,
               'm_bert_lcoh': np.mean(self.local_coherence_list(model='bert')),
               'm_bert_gcoh': np.mean(self.global_coherence_list(model='bert')),
               'm_bert_cgcoh': np.mean(self.cumulative_global_coherence_list(model='bert')),
               'm_bert_scoh': np.mean(self.second_order_coherence_list(model='bert'))
               }


  # lexical

  def TTR(self, lemmas: bool = True) -> float:
    items = self.data.lemmas if lemmas else self.data.words
    return TTR(items)

  def MATTR(self, w: Optional[int] = 10, lemmas: bool = True) -> float:
    items = self.data.lemmas if lemmas else self.data.words
    return MATTR(items, w=w)

  # graph

  def graph_statistics(self, w: Optional[int] = 10, lemmas: bool = True) -> Dict[str, float]:
    items = self.data.lemmas if lemmas else self.data.words
    return moving_graph_statistics(items, w=w)

  # LM
  @cached_property
  def bert_sent_vectors(self) -> List[np.array]:
    return [bert_sent_vectorize(s,
                                model=self.bert_model,
                                tokenizer=self.bert_tokenizer)
    for s in data.sents]

  def sent_vectors(self, model: Optional[str] = 'default') -> List[np.array]:
    if model == 'bert':
      return self.bert_sent_vectors
    else:
      return self.data.sent_vectors

  def local_coherence_list(self, model: Optional[str] = 'default') -> List[float]:
    return get_local_coherence_list(self.sent_vectors(model=model))

  def global_coherence_list(self, model: Optional[str] = 'default') -> List[float]:
    return get_global_coherence_list(self.sent_vectors(model=model))

  def cumulative_global_coherence_list(self, model: Optional[str] = 'default') -> List[float]:
    return get_cumulative_global_coherence_list(self.sent_vectors(model=model))

  def second_order_coherence_list(self, model: Optional[str] = 'default') -> List[float]:
    return get_second_order_coherence_list(self.sent_vectors(model=model))

  @cached_property
  def sent_prob_list(self):
    return get_prob_list(self.data.sents,
                         model_nsp=self.bert_model_nsp,
                         tokenizer_nsp=self.bert_tokenizer_nsp)

  # values
  @cached_property
  def values(self):
    return {'LM': self.LM, 'syntactic': self.syntactic,
            'lexical': self.lexical, 'graph': self.graph_statistics}

In [79]:
d = ProcessTextData(data)
d.values

{'LM': {'m_lcoh': 0.4842664306268148,
  'm_gcoh': 0.7365893185018518,
  'm_cgcoh': 0.7693006449156451,
  'm_scoh': 0.5764330740012963,
  'm_sporb': 0.9885238707065582,
  'm_bert_lcoh': 0.8709225,
  'm_bert_gcoh': 0.9313899,
  'm_bert_cgcoh': 0.9489261,
  'm_bert_scoh': 0.8724599},
 'syntactic': {'n_sents': 7,
  'mean_sent_words': 28.857142857142858,
  'DET': 0.06930693069306931,
  'NOUN': 0.18316831683168316,
  'VERB': 0.04455445544554455,
  'PRON': 0.024752475247524754,
  'ADP': 0.09900990099009901,
  'ADJ': 0.07425742574257425,
  'PUNCT': 0.18316831683168316,
  'CCONJ': 0.0594059405940594,
  'AUX': 0.04950495049504951,
  'ADV': 0.08415841584158416,
  'PROPN': 0.06930693069306931,
  'SCONJ': 0.0049504950495049506,
  'NUM': 0.034653465346534656,
  'PART': 0.01485148514851485,
  'X': 0.0049504950495049506},
 'lexical': {'n_words': 202,
  'LTR': 0.5643564356435643,
  'MALTR': 0.8822916666666668},
 'graph': {'number_of_nodes': 8.822916666666666,
  'number_of_edges': 9.0,
  'PE': 0.0416666

In [80]:
d_ru = ProcessTextData(data_ru, bert_lang='ru')
d_ru.values

{'LM': {'m_lcoh': 0.999392411995775,
  'm_gcoh': 0.9998244997103022,
  'm_cgcoh': 0.9999012745447248,
  'm_scoh': 0.999507581413138,
  'm_sporb': 0.6925834740201632,
  'm_bert_lcoh': 0.97850305,
  'm_bert_gcoh': 0.9868716,
  'm_bert_cgcoh': 0.99049085,
  'm_bert_scoh': 0.97659856},
 'syntactic': {'n_sents': 4,
  'mean_sent_words': 18.0,
  'VERB': 0.09722222222222222,
  'PART': 0.041666666666666664,
  'PRON': 0.05555555555555555,
  'PUNCT': 0.19444444444444445,
  'ADJ': 0.1527777777777778,
  'NOUN': 0.19444444444444445,
  'ADP': 0.09722222222222222,
  'PROPN': 0.06944444444444445,
  'AUX': 0.027777777777777776,
  'NUM': 0.013888888888888888,
  'SCONJ': 0.013888888888888888,
  'CCONJ': 0.013888888888888888,
  'ADV': 0.027777777777777776},
 'lexical': {'n_words': 72, 'LTR': 0.75, 'MALTR': 0.9564516129032259},
 'graph': {'number_of_nodes': 9.564516129032258,
  'number_of_edges': 9.0,
  'PE': 0.0,
  'LCC': 9.564516129032258,
  'LSC': 2.0,
  'degree_average': 1.8903225806451611,
  'degree_st

In [ ]:
def average_values(list_of_values_dicts: List[Dict[str, Dict[str, float]]]) -> Dict[str, Dict[str, float]]:
  new_val = {metric_type: {metric_name: [] for metric_name in metric_dict.keys()}
             for metric_type, metric_dict in list_of_values_dicts[0].items()}
  for values in list_of_values_dicts:
    for metric_type, d in values.items():
      for metric_name, metric_value in d.items():
        if metric_name not in new_val[metric_type]:
          new_val[metric_type][metric_name] = [metric_value]
        new_val[metric_type][metric_name].append(metric_value)

  return {metric_type: {metric_name: np.mean(metric_values) for metric_name,
                        metric_values in metric_dict.items()}
             for metric_type, metric_dict in new_val.items()}

Test

In [ ]:
import pandas as pd
df = pd.read_csv('turns.tsv', sep='\t', index_col=0)

In [ ]:
newrows = {}
for index, row in df.iterrows():
  vals = []
  for w in row:
    v = ProcessTextData(process_spacy(w)).values
    vals.append(v)
  newrows[index] = average_values(vals)
  # print(row['wut_patient_turns'])

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
for_pd = []
for i, d in newrows.items():
  d_p = {'index' : i}
  for dd in d.values():
    d_p.update(dd)
  for_pd.append(d_p)
new_df = pd.DataFrame(for_pd)
with open('NET_NAP_new.tsv', 'w') as f:
  f.write(new_df.to_csv(index=False, sep='\t'))

In [ ]:
new_df.head()

,index,m_lcoh,m_gcoh,m_cgcoh,m_scoh,m_sporb,m_bert_lcoh,m_bert_gcoh,m_bert_cgcoh,m_bert_scoh,...,PE,LCC,LSC,degree_average,degree_std,L1,L2,L3,X,PROPN
0,34,0.445511,0.665204,0.713319,0.480528,0.855603,0.870923,0.93139,0.948926,0.87246,...,0.203084,8.375699,3.944242,2.219160,1.001812,0.426063,1.732931,3.689045,NaN,NaN
1,60,0.417252,0.615969,0.678460,0.321578,0.887510,0.870923,0.93139,0.948926,0.87246,...,0.183619,8.908038,3.571602,2.059414,0.675319,0.122953,0.604877,1.239897,0.008213,0.019545
2,76,0.376666,0.607762,0.663830,0.356495,0.859433,0.870923,0.93139,0.948926,0.87246,...,0.183220,8.576408,4.232904,2.147339,0.908804,0.274941,1.068746,2.936122,0.004237,0.012254
3,84,0.334272,0.603192,0.669950,0.357711,0.883172,0.870923,0.93139,0.948926,0.87246,...,0.141174,8.763347,3.989202,2.093529,0.764162,0.069488,0.727639,1.383873,0.016592,0.010665
4,88,0.711440,0.837232,0.881330,0.489078,0.950917,0.870923,0.93139,0.948926,0.87246,...,0.039770,8.903360,4.372477,2.040719,0.648437,0.006503,0.106605,0.435152,0.002809,0.026404
